# Implementations

This notebook delves into the world of ECC implementations.

 - You will first [analyze traces](#Manual-analysis) collected from a Curve25519 implementation to learn information about the implementation.
    - How many iterations are in the scalar multiplier?
    - Is the scalar multiplier left-to-right or right-to-left?
 - Then you will explore the number of ECC implementations [combinatorially](#Implementation-space).

## Manual analysis

[sca25519](https://github.com/sca-secure-library-sca25519/sca25519) is an open-source implementation of the Curve25519 key-exchange (XDH) for the ARM Cortex-M4. It contains three **implementations**:

 - unprotected
 - ephemeral
 - static

that contain different countermeasures, with the unprotected one being free of any.

You will work with three trace sets of scalar multiplication:
 - **A**: [Download](https://neuromancer.sk/static/traces_A.pickle) Traces of the full scalar multiplication. 10 traces per implementation: all implementations. Fixed scalar and point.
 - **B**: [Download](https://neuromancer.sk/static/traces_B.pickle) Traces of the beginning of the scalar multiplication. 1000 traces per group: random point group, random scalar group. Unprotected implementation.
 - **C**: [Download](https://neuromancer.sk/static/traces_C.pickle) Traces of the end of the scalar multiplication. 1000 traces. Unprotected implementation. Random scalar. 

In [ ]:
from pyecsca.sca.trace_set import PickleTraceSet, HDF5TraceSet
from pyecsca.sca.trace import Trace
from pyecsca.ec.params import get_params
from pyecsca.ec.point import Point
from pyecsca.ec.mod import Mod

import holoviews as hv
import numpy as np

In [ ]:
hv.extension("bokeh")
%opts RGB [height=600, responsive=True]

In [ ]:
curve25519 = get_params("other", "Curve25519", "xz")
p = curve25519.curve.prime
n = curve25519.order

### <span style="color:#00468C; font-weight: bold;">Exercise</span>

Use trace set **A** and visually analyze the traces:

 - Plot them.
 - Compare them between implementations.
 - Process them using e.g. rolling mean and count peaks on them.
 - How many iterations does the scalar multiplier have in the unprotected case?

**Docs**<br/>
[plot module](https://neuromancer.sk/pyecsca/api/pyecsca.sca.trace.plot.html)<br/>
[scipy.signal.find_peaks](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html)

In [ ]:
from pyecsca.sca.trace.plot import plot_trace, plot_traces
from pyecsca.sca.trace.process import rolling_mean
from scipy.signal import find_peaks

def plot_trace_peaks(peaks, trace):
    return plot_trace(trace) * hv.Points((peaks, trace.samples[peaks])).opts(color="red")

In [ ]:
traces_A = PickleTraceSet.read("../traces_A.pickle")

In [ ]:
traces_A[0]

In [ ]:
# Task: Plot two traces from the unprotected implementation.
# Hint: Look at the .meta attribute of the traces.
plot_traces(traces_A[0], traces_A[1])

In [ ]:
# Task: Plot traces from the other implementations.
plot_traces(traces_A[10])

In [ ]:
plot_traces(traces_A[20])

In [ ]:
# Task: Use rolling_mean and find_peaks to count the iterations in the unprotected implementation
# Note: Before applying the rolling mean, make sure to transform the dtype of the trace by doing trace.astype(np.float32) and using the result.
# Hint: You can use plot_trace_peaks to plot the trace along with the detected peaks from find_peaks.
meany = rolling_mean(traces_A[0].astype(np.float32), 5000)
peaks, other = find_peaks(meany.samples, height=10, distance=5000)
print(len(peaks))
plot_trace_peaks(peaks, meany)

### <span style="color:#00468C; font-weight: bold;">Exercise</span>

Use trace sets **B** and **C** and correlate the bits at the beginning and end of the scalar with the trace sets (respectively).
This will help you figure out whether the multiplier is left-to-right or right-to-left.


**Docs**<br/>
TODO

## Implementation space
TODO

In [ ]:
from typing import get_args
from pyecsca.ec.configuration import Configuration
from dataclasses import fields

for field in fields(Configuration):
    name = field.name
    tp = field.type
    doc = tp.__doc__
    if get_args(field.type):
            doc = get_args(field.type)[0].__doc__
    print(name, tp)
    print("   ", doc)
    if hasattr(tp, "names"):
        for enum_name in tp.names():
            print("       ", enum_name)
    print()

In [ ]:
from pyecsca.ec.configuration import all_configurations, HashType, RandomMod, Multiplication, Squaring, Reduction, Inversion
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.mult import LTRMultiplier

model = ShortWeierstrassModel()
coords = model.coordinates["projective"]
scalarmult = LTRMultiplier
independent_opts = {
    "hash_type": HashType.SHA256,
    "mod_rand": RandomMod.SAMPLE,
    "mult": Multiplication.KARATSUBA,
    "sqr": Squaring.KARATSUBA,
    "red": Reduction.MONTGOMERY,
    "inv": Inversion.GCD
}

configs = list(all_configurations(model=model, coords=coords, scalarmult=scalarmult,
                                                              **independent_opts))
print(len(configs))

In [ ]:
from IPython.display import HTML, display
import tabulate
from pyecsca.ec.model import *
from pyecsca.ec.mult import ProcessingDirection, AccumulationOrder

no_indep = (6*2*4*4*3*2)
no_ff = (6*2)

model_counts = [["Model", "All", "Without independent options", "Without independent options and scaling", "Without independent options and scalarmult options"]]
totals = ["Total", 0, 0, 0, 0]
for model in (ShortWeierstrassModel(), MontgomeryModel(), EdwardsModel(), TwistedEdwardsModel()):
    name = model.__class__.__name__
    count = sum(1 for _ in all_configurations(model=model, **independent_opts))
    count_no_scl = sum(1 for _ in all_configurations(model=model, **independent_opts, scalarmult={"scl": None}))
    count_no_opts = sum(1 for _ in all_configurations(model=model, **independent_opts, scalarmult={"scl": None, "always": True, "short_circuit": True, "complete": False, "precompute_negation": True, "width": 3, "m": 3, "direction": ProcessingDirection.LTR, "accumulation_order": AccumulationOrder.PeqPR, "recoding_direction": ProcessingDirection.LTR}))
    model_counts.append([name, count * no_ff, count, count_no_scl, count_no_opts])
    totals[1] += count * no_ff
    totals[2] += count
    totals[3] += count_no_scl
    totals[4] += count_no_opts
model_counts.append(totals)
display(HTML(tabulate.tabulate(model_counts, tablefmt="html", headers="firstrow")))

In [ ]:
coords_counts = [["Model", "Coords", "All", "Without independent options", "Without independent options and scaling", "Without independent options and scalarmult options"]]
for model in (ShortWeierstrassModel(), MontgomeryModel(), EdwardsModel(), TwistedEdwardsModel()):
    model_name = model.__class__.__name__
    coords_counts.append([model_name, "", "", "", "", ""])
    for coords in sorted(model.coordinates.values(), key=lambda c: c.name):
            coords_name = coords.name
            count = sum(1 for _ in all_configurations(model=model, coords=coords, **independent_opts))
            count_no_scl = sum(1 for _ in all_configurations(model=model, coords=coords, **independent_opts, scalarmult={"scl": None}))
            count_no_opts = sum(1 for _ in all_configurations(model=model, coords=coords, **independent_opts, scalarmult={"scl": None, "always": True, "short_circuit": True, "complete": False, "precompute_negation": True, "width": 3, "m": 3, "direction": ProcessingDirection.LTR, "accumulation_order": AccumulationOrder.PeqPR, "recoding_direction": ProcessingDirection.LTR}))
            coords_counts.append(["", coords_name, count * no_ff, count, count_no_scl, count_no_opts])

display(HTML(tabulate.tabulate(coords_counts, tablefmt="html", headers="firstrow")))

In [ ]:
from pyecsca.ec.mult import ScalarMultiplier

def leaf_subclasses(cls):
    subs = cls.__subclasses__()
    result = set()
    for subclass in subs:
        if subclass.__subclasses__():
            result.update(leaf_subclasses(subclass))
        else:
            result.add(subclass)
    return result

mult_counts = [["ScalarMultiplier", "All", "Without independent options", "Without independent options and scaling", "Without independent options and scalarmult options"]]
for mult_cls in leaf_subclasses(ScalarMultiplier):
    count = sum(1 for _ in all_configurations(**independent_opts, scalarmult=mult_cls))
    count_no_scl = sum(1 for _ in all_configurations(**independent_opts, scalarmult={"cls": mult_cls, "scl": None}))
    count_no_opts = sum(1 for _ in all_configurations(**independent_opts, scalarmult={"cls": mult_cls, "scl": None, "always": True, "short_circuit": True, "complete": False, "precompute_negation": True, "width": 3, "m": 3, "direction": ProcessingDirection.LTR, "accumulation_order": AccumulationOrder.PeqPR, "recoding_direction": ProcessingDirection.LTR}))
    mult_counts.append([mult_cls.__name__, count * no_ff, count, count_no_scl, count_no_opts])

display(HTML(tabulate.tabulate(mult_counts, tablefmt="html", headers="firstrow")))